###  变长序列的处理方式

变长序列的处理逻辑就是：

👉 虽然原始的变长序列（如 genres、keywords）在输入时是一个序列，但在经过 embedding 和 pooling（这里是 mean pooling）之后，会被压缩成一个定长向量，相当于一个“整体语义”表示。
换句话说：

原始序列（如 ['动作', '冒险', '科幻']）
⬇️
通过 Embedding → 变成 (batch_size, seq_len, embedding_dim) 的张量
⬇️
通过 mean pooling → 汇总成 (batch_size, 1, embedding_dim)
⬇️
和其它稀疏特征一起参与拼接、建模
所以最终：

无论是稀疏特征（单个索引）还是变长特征（序列），最终都统一成 (batch_size, 1, embedding_dim) 的形式，这样才能拼接起来喂给 FM 和 DNN。

这种做法其实就是“把变长特征转化为定长特征”的通用套路之一（还有比如 attention pooling、CNN、RNN 都可以做类似事情）。

###  整体程序

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer

# 1. 加载并预处理数据，这里模拟出2个变长序列数据
data = pd.read_csv(r"D:\software\pycharm_repository\StarMaker\MultiRecSys\data_files\movielens_sample.txt")
data['genres_bak'] = data['genres']
data.head()

sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ["rating"]
sequence_features = ["genres", "genres_bak"]

# 对稀疏特征做标签编码（Label Encoding）
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

# 用于存储每个变长特征处理后的 padding 序列
pad_sequences_dict = {}

# 每个变长特征对应一个独立的 Tokenizer，用于后续文本转索引
tokenizers = {}

# 用于记录每个变长特征的 padding 长度（即序列被填充后的最大长度）
pad_len_dict = {}

# 遍历所有变长序列特征
for feature in sequence_features:
    texts = data[feature].apply(lambda x: x.replace('|', ' ')).tolist()
    tokenizer = Tokenizer(oov_token='OOV')
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, padding='post')  # shape: (num_samples, max_seq_len)
    pad_sequences_dict[feature] = padded
    tokenizers[feature] = tokenizer
    pad_len_dict[feature] = padded.shape[1]  # 保存每个特征的序列长度（max_seq_len）

# 2. 创建所有特征的Embedding层
embedding_dim = 4
vocab_sizes = {feat: data[feat].nunique() for feat in sparse_features}

for feature in sequence_features:
    feat_num = len(tokenizers[feature].word_index) + 1
    vocab_sizes[feature] = feat_num

# 创建嵌入层字典
embed_layers = {}
for feat in sparse_features:
    embed_layers[feat] = layers.Embedding(input_dim=vocab_sizes[feat] + 1, output_dim=embedding_dim, mask_zero=False)
for feat in sequence_features:
    embed_layers[feat] = layers.Embedding(input_dim=vocab_sizes[feat] + 1, output_dim=embedding_dim, mask_zero=True)

# 1. 初始化模型输入字典
inputs = {}
for feat in sparse_features:
    inputs[feat] = tf.keras.Input(shape=(1,), name=feat, dtype=tf.int32)  # shape: (batch_size, 1)
for feat in sequence_features:
    max_len = pad_len_dict[feat]
    inputs[feat] = tf.keras.Input(shape=(max_len,), name=feat, dtype=tf.int32)  # shape: (batch_size, max_len)

# 2. 构建嵌入列表
embeds = []
for feat in sparse_features:
    embed = embed_layers[feat](inputs[feat])  # shape: (batch_size, 1, embedding_dim)
    embeds.append(embed)

for feat in sequence_features:
    seq_embed = embed_layers[feat](inputs[feat])  # shape: (batch_size, seq_len, embedding_dim)
    pooled_embed = tf.reduce_mean(seq_embed, axis=1, keepdims=True)  # shape: (batch_size, 1, embedding_dim) 从这可以看出边长序列的字段数据最终整体也是当成一个字段处理
    embeds.append(pooled_embed)

# 拼接所有嵌入特征
total_embeds = tf.concat(embeds, axis=1)  # shape: (batch_size, num_fields, embedding_dim)

# 4. FM 二阶交叉项计算
sum_square = tf.square(tf.reduce_sum(total_embeds, axis=1))  # shape: (batch_size, embedding_dim)
square_sum = tf.reduce_sum(tf.square(total_embeds), axis=1)  # shape: (batch_size, embedding_dim)
fm_second_order = 0.5 * tf.reduce_sum(sum_square - square_sum, axis=1, keepdims=True)  # shape: (batch_size, 1)

# 5. DNN 部分
flatten_input = tf.reshape(total_embeds, shape=(-1, total_embeds.shape[1] * embedding_dim))  # shape: (batch_size, num_fields * embedding_dim)
x = layers.Dense(64, activation='relu')(flatten_input)  # shape: (batch_size, 64)
x = layers.Dense(32, activation='relu')(x)              # shape: (batch_size, 32)
dnn_output = layers.Dense(1)(x)                         # shape: (batch_size, 1)

# 6. 合并 FM 和 DNN 输出结果
output = layers.Add()([fm_second_order, dnn_output])    # shape: (batch_size, 1)
model = tf.keras.Model(inputs=list(inputs.values()), outputs=output)

# 7. 编译并训练模型
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# 构建模型输入字典
model_input = {}
for feat in sparse_features:
    model_input[feat] = data[feat].values  # shape: (num_samples,)
for feat in sequence_features:
    model_input[feat] = pad_sequences_dict[feat]  # shape: (num_samples, max_seq_len)

# 模型训练
model.fit(model_input, data[target].values, batch_size=256, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 14.2088 - mse: 14.2088 - val_loss: 13.3243 - val_mse: 13.3243
Epoch 2/10
1/1 [==============================] - 0s 37ms/step - loss: 14.1188 - mse: 14.1188 - val_loss: 13.2393 - val_mse: 13.2393
Epoch 3/10
1/1 [==============================] - 0s 41ms/step - loss: 14.0249 - mse: 14.0249 - val_loss: 13.1507 - val_mse: 13.1507
Epoch 4/10
1/1 [==============================] - 0s 37ms/step - loss: 13.9267 - mse: 13.9267 - val_loss: 13.0584 - val_mse: 13.0584
Epoch 5/10
1/1 [==============================] - 0s 33ms/step - loss: 13.8235 - mse: 13.8235 - val_loss: 12.9604 - val_mse: 12.9604
Epoch 6/10
1/1 [==============================] - 0s 33ms/step - loss: 13.7151 - mse: 13.7151 - val_loss: 12.8573 - val_mse: 12.8573
Epoch 7/10
1/1 [==============================] - 0s 37ms/step - loss: 13.6011 - mse: 13.6011 - val_loss: 12.7488 - val_mse: 12.7488
Epoch 8/10
1/1 [==============================] - 0s 33ms/step - loss: 

### 详细注释

In [4]:
# 1. 加载并预处理数据，这里模拟出2个变长序列数据
data = pd.read_csv(r"D:\software\pycharm_repository\StarMaker\MultiRecSys\data_files\movielens_sample.txt")
data['genres_bak'] = data['genres']
data.head()

user_id  movie_id  rating  timestamp  \
0     3299       235       4  968035345   
1     3630      3256       3  966536874   
2      517       105       4  976203603   
3      785      2115       3  975430389   
4     5848       909       5  957782527   

                                         title            genres gender  age  \
0                               Ed Wood (1994)      Comedy|Drama      F   25   
1                         Patriot Games (1992)   Action|Thriller      M   18   
2        Bridges of Madison County, The (1995)     Drama|Romance      F   25   
3  Indiana Jones and the Temple of Doom (1984)  Action|Adventure      M   18   
4                        Apartment, The (1960)      Comedy|Drama      M   50   

   occupation    zip        genres_bak  
0           4  19119      Comedy|Drama  
1           4  77005   Action|Thriller  
2          14  55408     Drama|Romance  
3          19  29307  Action|Adventure  
4          20  20009      Comedy|Drama

In [5]:
sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
target = ["rating"]
sequence_features = ["genres", "genres_bak"]

In [6]:
# 对稀疏特征做标签编码（Label Encoding）
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [7]:
# 用于存储每个变长特征处理后的 padding 序列
pad_sequences_dict = {}

# 每个变长特征对应一个独立的 Tokenizer，用于后续文本转索引
tokenizers = {}

# 用于记录每个变长特征的 padding 长度（即序列被填充后的最大长度）
pad_len_dict = {}

In [8]:
# 遍历所有变长序列特征
for feature in sequence_features:
    texts = data[feature].apply(lambda x: x.replace('|', ' ')).tolist()
    tokenizer = Tokenizer(oov_token='OOV')
    tokenizer.fit_on_texts(texts)
    sequences = tokenizer.texts_to_sequences(texts)
    padded = pad_sequences(sequences, padding='post')  # shape: (num_samples, max_seq_len)
    pad_sequences_dict[feature] = padded
    tokenizers[feature] = tokenizer
    pad_len_dict[feature] = padded.shape[1]  # 保存每个特征的序列长度（max_seq_len）

In [9]:
pad_sequences_dict

{'genres': array([[2, 3, 0, 0, 0, 0],
        [4, 5, 0, 0, 0, 0],
        [3, 6, 0, 0, 0, 0],
        ...,
        [2, 6, 0, 0, 0, 0],
        [4, 9, 5, 0, 0, 0],
        [2, 0, 0, 0, 0, 0]]),
 'genres_bak': array([[2, 3, 0, 0, 0, 0],
        [4, 5, 0, 0, 0, 0],
        [3, 6, 0, 0, 0, 0],
        ...,
        [2, 6, 0, 0, 0, 0],
        [4, 9, 5, 0, 0, 0],
        [2, 0, 0, 0, 0, 0]])}

In [10]:
tokenizers

{'genres': <keras_preprocessing.text.Tokenizer at 0x1830946b730>,
 'genres_bak': <keras_preprocessing.text.Tokenizer at 0x182ed0018e0>}

In [11]:
pad_len_dict

{'genres': 6, 'genres_bak': 6}

In [12]:
# 2. 创建所有特征的Embedding层
embedding_dim = 4
vocab_sizes = {feat: data[feat].nunique() for feat in sparse_features}

for feature in sequence_features:
    feat_num = len(tokenizers[feature].word_index) + 1
    vocab_sizes[feature] = feat_num

In [13]:
vocab_sizes

{'movie_id': 187,
 'user_id': 193,
 'gender': 2,
 'age': 7,
 'occupation': 20,
 'zip': 188,
 'genres': 21,
 'genres_bak': 21}

In [14]:
# 创建嵌入层字典
embed_layers = {}
for feat in sparse_features:
    embed_layers[feat] = layers.Embedding(input_dim=vocab_sizes[feat] + 1, output_dim=embedding_dim, mask_zero=False)
for feat in sequence_features:
    embed_layers[feat] = layers.Embedding(input_dim=vocab_sizes[feat] + 1, output_dim=embedding_dim, mask_zero=True)

In [15]:
embed_layers

{'movie_id': <keras.layers.embeddings.Embedding at 0x1830946f7c0>,
 'user_id': <keras.layers.embeddings.Embedding at 0x1830946f910>,
 'gender': <keras.layers.embeddings.Embedding at 0x1830946fb80>,
 'age': <keras.layers.embeddings.Embedding at 0x1830946feb0>,
 'occupation': <keras.layers.embeddings.Embedding at 0x1830946f9a0>,
 'zip': <keras.layers.embeddings.Embedding at 0x1830946bcd0>,
 'genres': <keras.layers.embeddings.Embedding at 0x1830946f250>,
 'genres_bak': <keras.layers.embeddings.Embedding at 0x1830946bc10>}

In [16]:
# 1. 初始化模型输入字典
inputs = {}
for feat in sparse_features:
    inputs[feat] = tf.keras.Input(shape=(1,), name=feat, dtype=tf.int32)  # shape: (batch_size, 1)
for feat in sequence_features:
    max_len = pad_len_dict[feat]
    inputs[feat] = tf.keras.Input(shape=(max_len,), name=feat, dtype=tf.int32)  # shape: (batch_size, max_len)
inputs

{'movie_id': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'movie_id')>,
 'user_id': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_id')>,
 'gender': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'gender')>,
 'age': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'age')>,
 'occupation': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'occupation')>,
 'zip': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'zip')>,
 'genres': <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres')>,
 'genres_bak': <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres_bak')>}

In [17]:
# 2. 构建嵌入列表
embeds = []
for feat in sparse_features:
    embed = embed_layers[feat](inputs[feat])  # shape: (batch_size, 1, embedding_dim)
    embeds.append(embed)

for feat in sequence_features:
    seq_embed = embed_layers[feat](inputs[feat])  # shape: (batch_size, seq_len, embedding_dim)
    pooled_embed = tf.reduce_mean(seq_embed, axis=1, keepdims=True)  # shape: (batch_size, 1, embedding_dim) 从这可以看出边长序列的字段数据最终整体也是当成一个字段处理
    embeds.append(pooled_embed)

In [18]:
embeds

[<KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_8')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_9')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_10')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_11')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_12')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'embedding_13')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'tf.math.reduce_mean_2')>,
 <KerasTensor: shape=(None, 1, 4) dtype=float32 (created by layer 'tf.math.reduce_mean_3')>]

In [19]:
# 拼接所有嵌入特征
total_embeds = tf.concat(embeds, axis=1)  # shape: (batch_size, num_fields, embedding_dim)
total_embeds

<KerasTensor: shape=(None, 8, 4) dtype=float32 (created by layer 'tf.concat_1')>

In [20]:
# 4. FM 二阶交叉项计算
sum_square = tf.square(tf.reduce_sum(total_embeds, axis=1))  # shape: (batch_size, embedding_dim)
square_sum = tf.reduce_sum(tf.square(total_embeds), axis=1)  # shape: (batch_size, embedding_dim)
fm_second_order = 0.5 * tf.reduce_sum(sum_square - square_sum, axis=1, keepdims=True)  # shape: (batch_size, 1)

In [21]:
# 5. DNN 部分
flatten_input = tf.reshape(total_embeds, shape=(-1, total_embeds.shape[1] * embedding_dim))  # shape: (batch_size, num_fields * embedding_dim)
x = layers.Dense(64, activation='relu')(flatten_input)  # shape: (batch_size, 64)
x = layers.Dense(32, activation='relu')(x)              # shape: (batch_size, 32)
dnn_output = layers.Dense(1)(x)                         # shape: (batch_size, 1)

In [22]:
# 6. 合并 FM 和 DNN 输出结果
output = layers.Add()([fm_second_order, dnn_output])    # shape: (batch_size, 1)
output

<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'add_1')>

In [23]:
model = tf.keras.Model(inputs=list(inputs.values()), outputs=output)

In [24]:
model

In [25]:
inputs

{'movie_id': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'movie_id')>,
 'user_id': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_id')>,
 'gender': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'gender')>,
 'age': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'age')>,
 'occupation': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'occupation')>,
 'zip': <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'zip')>,
 'genres': <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres')>,
 'genres_bak': <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres_bak')>}

In [26]:
inputs.values()

dict_values([<KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'movie_id')>, <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_id')>, <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'gender')>, <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'age')>, <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'occupation')>, <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'zip')>, <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres')>, <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres_bak')>])

In [27]:
inputs=list(inputs.values())
inputs

[<KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'movie_id')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'user_id')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'gender')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'age')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'occupation')>,
 <KerasTensor: shape=(None, 1) dtype=int32 (created by layer 'zip')>,
 <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres')>,
 <KerasTensor: shape=(None, 6) dtype=int32 (created by layer 'genres_bak')>]

In [28]:
# 7. 编译并训练模型
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# 构建模型输入字典
model_input = {}
for feat in sparse_features:
    model_input[feat] = data[feat].values  # shape: (num_samples,)
for feat in sequence_features:
    model_input[feat] = pad_sequences_dict[feat]  # shape: (num_samples, max_seq_le

In [29]:
model_input

{'movie_id': array([ 12, 169,   6, 112,  45, 146,  43, 156,  30, 174,  82, 173,  91,
        108, 132,  40, 109,  31, 180, 183, 129,  67, 137,  87, 127,   8,
        104, 100, 140,  25, 122, 124, 116, 126,  72, 117,  42, 145, 131,
          2,  52,  17, 101,  94, 136,  65,  20, 144,  26,  83,  55, 126,
        184,  23, 121, 142,  33,   0,  46, 139, 150, 135,  36, 110,  79,
        162,  70, 147,   9,  34,   7,  76,   4, 185,  73, 112, 130,  95,
         28,  24, 148, 119, 168, 149, 181,  13, 154,  56,  66, 172,  69,
         35,  49, 106,  35,  11, 152, 166,  37, 164,  54, 167,  72,  29,
         92, 114,  88, 170,  64,  60,  38,  22,  62, 178, 134, 157,  99,
         34, 111,  96,  50,  75,  47,  14,  21,  77, 118, 182, 113, 143,
        149, 141,  10,  58,  81,  44,  27, 151, 165,  98, 163,  80, 158,
        161,  27, 155, 171,  78,  57, 123,  84,  93, 170, 120,   1, 153,
         39,  61,  51,  71,  19, 107,   9,  66, 102,  74, 177, 103, 133,
        160,  53,  90,   5, 173,  41,  

In [30]:
# 模型训练
model.fit(model_input, data[target].values, batch_size=256, epochs=10, validation_split=0.2)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 14.3072 - mse: 14.3072 - val_loss: 13.4602 - val_mse: 13.4602
Epoch 2/10
1/1 [==============================] - 0s 36ms/step - loss: 14.2007 - mse: 14.2007 - val_loss: 13.3670 - val_mse: 13.3670
Epoch 3/10
1/1 [==============================] - 0s 36ms/step - loss: 14.0969 - mse: 14.0969 - val_loss: 13.2767 - val_mse: 13.2767
Epoch 4/10
1/1 [==============================] - 0s 36ms/step - loss: 13.9932 - mse: 13.9932 - val_loss: 13.1862 - val_mse: 13.1862
Epoch 5/10
1/1 [==============================] - 0s 37ms/step - loss: 13.8870 - mse: 13.8870 - val_loss: 13.0909 - val_mse: 13.0909
Epoch 6/10
1/1 [==============================] - 0s 36ms/step - loss: 13.7778 - mse: 13.7778 - val_loss: 12.9928 - val_mse: 12.9928
Epoch 7/10
1/1 [==============================] - 0s 37ms/step - loss: 13.6647 - mse: 13.6647 - val_loss: 12.8905 - val_mse: 12.8905
Epoch 8/10
1/1 [==============================] - 0s 37ms/step - loss: 

### 模型封装为类的版本

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers, models
import tensorflow as tf


class FeatureProcessor:
    def __init__(self, sparse_features, sequence_features, embedding_dim=4):
        self.sparse_features = sparse_features
        self.sequence_features = sequence_features
        self.embedding_dim = embedding_dim

        self.label_encoders = {}
        self.tokenizers = {}
        self.pad_sequences_dict = {}
        self.pad_len_dict = {}
        self.vocab_sizes = {}

    def fit_transform(self, df):
        # 编码稀疏特征
        for feat in self.sparse_features:
            lbe = LabelEncoder()
            df[feat] = lbe.fit_transform(df[feat])
            self.label_encoders[feat] = lbe
            self.vocab_sizes[feat] = df[feat].nunique()

        # 编码变长序列特征
        for feat in self.sequence_features:
            texts = df[feat].apply(lambda x: x.replace('|', ' ')).tolist()
            tokenizer = Tokenizer(oov_token='OOV')
            tokenizer.fit_on_texts(texts)
            sequences = tokenizer.texts_to_sequences(texts)
            padded = pad_sequences(sequences, padding='post')
            self.tokenizers[feat] = tokenizer
            self.pad_sequences_dict[feat] = padded
            self.pad_len_dict[feat] = padded.shape[1]
            self.vocab_sizes[feat] = len(tokenizer.word_index) + 1

        return df

    def get_model_input(self, df):
        model_input = {}
        for feat in self.sparse_features:
            model_input[feat] = df[feat].values
        for feat in self.sequence_features:
            model_input[feat] = self.pad_sequences_dict[feat]
        return model_input


class FM_DNN_Model(tf.keras.Model):
    def __init__(self, sparse_features, sequence_features, vocab_sizes, pad_len_dict, embedding_dim=4):
        super(FM_DNN_Model, self).__init__()
        self.embedding_dim = embedding_dim
        self.sparse_features = sparse_features
        self.sequence_features = sequence_features
        self.vocab_sizes = vocab_sizes
        self.pad_len_dict = pad_len_dict

        self.embed_layers = {}
        for feat in sparse_features:
            self.embed_layers[feat] = layers.Embedding(input_dim=vocab_sizes[feat] + 1,
                                                       output_dim=embedding_dim,
                                                       mask_zero=False)

        for feat in sequence_features:
            self.embed_layers[feat] = layers.Embedding(input_dim=vocab_sizes[feat] + 1,
                                                       output_dim=embedding_dim,
                                                       mask_zero=True)

        # DNN 部分
        self.dnn = tf.keras.Sequential([
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(1)
        ])

    def call(self, inputs):
        embeds = []

        # 稀疏特征嵌入 shape: (batch, 1, embed_dim)
        for feat in self.sparse_features:
            embed = self.embed_layers[feat](inputs[feat])
            embeds.append(embed)

        # 序列特征嵌入并池化 shape: (batch, 1, embed_dim)
        for feat in self.sequence_features:
            embed = self.embed_layers[feat](inputs[feat])
            pooled = tf.reduce_mean(embed, axis=1, keepdims=True)
            embeds.append(pooled)

        print(embeds)
        
        total_embeds = tf.concat(embeds, axis=1)  # shape: (batch, field_num, embed_dim)

        # FM 二阶交叉项
        sum_square = tf.square(tf.reduce_sum(total_embeds, axis=1))
        square_sum = tf.reduce_sum(tf.square(total_embeds), axis=1)
        fm_output = 0.5 * tf.reduce_sum(sum_square - square_sum, axis=1, keepdims=True)

        # DNN 输入
        flatten_input = tf.reshape(total_embeds, shape=(-1, total_embeds.shape[1] * self.embedding_dim))
        dnn_output = self.dnn(flatten_input)

        # FM + DNN
        return fm_output + dnn_output


# =================== 示例使用 ===================
if __name__ == '__main__':
    # 读取数据
    data = pd.read_csv(r"D:\software\pycharm_repository\StarMaker\MultiRecSys\data_files\movielens_sample.txt")
    data['genres_bak'] = data['genres']

    sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
    sequence_features = ["genres", "genres_bak"]
    target = ["rating"]

    # 特征处理
    processor = FeatureProcessor(sparse_features, sequence_features, embedding_dim=4)
    data = processor.fit_transform(data)
    model_input = processor.get_model_input(data)

    # 模型构建
    model = FM_DNN_Model(sparse_features, sequence_features,
                         vocab_sizes=processor.vocab_sizes,
                         pad_len_dict=processor.pad_len_dict,
                         embedding_dim=4)

    # 模型编译 & 训练
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    model.fit(model_input, data[target].values, batch_size=256, epochs=10, validation_split=0.2)

Epoch 1/10
[<tf.Tensor 'fm_dnn__model/embedding/embedding_lookup/Identity_1:0' shape=(None, 1, 4) dtype=float32>, <tf.Tensor 'fm_dnn__model/embedding_1/embedding_lookup/Identity_1:0' shape=(None, 1, 4) dtype=float32>, <tf.Tensor 'fm_dnn__model/embedding_2/embedding_lookup/Identity_1:0' shape=(None, 1, 4) dtype=float32>, <tf.Tensor 'fm_dnn__model/embedding_3/embedding_lookup/Identity_1:0' shape=(None, 1, 4) dtype=float32>, <tf.Tensor 'fm_dnn__model/embedding_4/embedding_lookup/Identity_1:0' shape=(None, 1, 4) dtype=float32>, <tf.Tensor 'fm_dnn__model/embedding_5/embedding_lookup/Identity_1:0' shape=(None, 1, 4) dtype=float32>, <tf.Tensor 'fm_dnn__model/Mean:0' shape=(None, 1, 4) dtype=float32>, <tf.Tensor 'fm_dnn__model/Mean_1:0' shape=(None, 1, 4) dtype=float32>]
[<tf.Tensor 'fm_dnn__model/embedding/embedding_lookup/Identity_1:0' shape=(None, 1, 4) dtype=float32>, <tf.Tensor 'fm_dnn__model/embedding_1/embedding_lookup/Identity_1:0' shape=(None, 1, 4) dtype=float32>, <tf.Tensor 'fm_dnn_